In [1]:
# prompt: install transformer library

!pip install transformers


In [2]:

import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import cross_val_score
import torch
import transformers as ppb
import warnings
warnings.filterwarnings('ignore')

In [3]:
df = pd.read_csv('https://github.com/clairett/pytorch-sentiment-classification/raw/master/data/SST2/train.tsv', delimiter='\t', header=None)

In [4]:
batch_1 = df[:200]

In [5]:
batch_1[1].value_counts()

1
1    113
0     87
Name: count, dtype: int64

In [6]:
# For BERT:
model_class, tokenizer_class, pretrained_weights = (ppb.BertModel, ppb.BertTokenizer, 'distilbert-base-cased')

## Want DistilBERT instead of distilBERT? Uncomment the following line:
#model_class, tokenizer_class, pretrained_weights = (ppb.DistilBertModel, ppb.DistilBertTokenizer, 'distilbert-base-uncased')


# Load pretrained model/tokenizer
tokenizer = tokenizer_class.from_pretrained(pretrained_weights)
model = model_class.from_pretrained(pretrained_weights)

The tokenizer class you load from this checkpoint is not the same type as the class this function is called from. It may result in unexpected tokenization. 
The tokenizer class you load from this checkpoint is 'DistilBertTokenizer'. 
The class this function is called from is 'BertTokenizer'.
You are using a model of type distilbert to instantiate a model of type bert. This is not supported for all configurations of models and can yield errors.


Some weights of BertModel were not initialized from the model checkpoint at distilbert-base-cased and are newly initialized: ['encoder.layer.8.attention.self.key.bias', 'pooler.dense.weight', 'pooler.dense.bias', 'encoder.layer.1.attention.self.key.weight', 'encoder.layer.2.intermediate.dense.bias', 'encoder.layer.7.attention.output.LayerNorm.weight', 'encoder.layer.2.intermediate.dense.weight', 'encoder.layer.10.attention.self.key.weight', 'encoder.layer.4.intermediate.dense.bias', 'encoder.layer.1.intermediate.dense.weight', 'encoder.layer.11.intermediate.dense.weight', 'encoder.layer.0.attention.output.dense.weight', 'encoder.layer.8.intermediate.dense.bias', 'encoder.layer.10.attention.self.value.bias', 'encoder.layer.9.intermediate.dense.weight', 'encoder.layer.1.attention.output.LayerNorm.weight', 'encoder.layer.9.output.dense.weight', 'encoder.layer.6.intermediate.dense.weight', 'encoder.layer.9.attention.self.query.weight', 'embeddings.token_type_embeddings.weight', 'encoder.la

In [7]:
tokenized = batch_1[0].apply((lambda x: tokenizer.encode(x, add_special_tokens=True)))

In [8]:
max_len = 0
for i in tokenized.values:
    if len(i) > max_len:
        max_len = len(i)

padded = np.array([i + [0]*(max_len-len(i)) for i in tokenized.values])

In [9]:
np.array(padded).shape

(200, 63)

In [10]:
attention_mask = np.where(padded != 0, 1, 0)
attention_mask.shape

(200, 63)

In [ ]:
input_ids = torch.tensor(padded)
attention_mask = torch.tensor(attention_mask)

with torch.no_grad():
    last_hidden_states = model(input_ids, attention_mask=attention_mask)

In [11]:
features = last_hidden_states[0][:,0,:].numpy()

NameError: name 'last_hidden_states' is not defined

In [ ]:
labels = batch_1[1]

In [ ]:
x_train, x_test, y_train, y_test = train_test_split(features, labels)

In [ ]:
lr_clf = LogisticRegression()
lr_clf.fit(x_train, y_train)

LogisticRegression()

In [ ]:
lr_clf.score(x_test, y_test)

0.85

In [ ]:
import numpy as np
pred= lr_clf.predict(x_test)

from sklearn.metrics import classification_report
print(classification_report(y_test,pred))

              precision    recall  f1-score   support

           0       0.84      0.87      0.85       247
           1       0.86      0.83      0.85       253

    accuracy                           0.85       500
   macro avg       0.85      0.85      0.85       500
weighted avg       0.85      0.85      0.85       500



In [ ]:
#https://learnopencv.com/fine-tuning-bert/

#https://kambale.dev/fine-tuning-bert

#https://wandb.ai/mukilan/BERT_Sentiment_Analysis/reports/An-Introduction-to-BERT-And-How-To-Use-It--VmlldzoyNTIyOTA1